In [ ]:
import numpy as np

from pyvo.dal import tap
from astropy.io import ascii

from tqdm import tqdm
import requests

In [ ]:
def get_targets(file, target_header='Star'):
    labels = ascii.read(file)
    return np.array(labels[target_header])

In [ ]:
tapobs = tap.TAPService("http://archive.eso.org/tap_obs")

def get_dpids( targets, count, fil ):
    
    dpids = []
    
    for target in tqdm(targets, desc="Retrieving DP ID's", leave=False):
        query = """SELECT TOP %i *
        FROM ivoa.ObsCore
        WHERE obs_collection='HARPS'
        AND target_name='%s'
        AND obs_release_date < getdate()
        ORDER BY snr DESC""" %(count, target)

        res = tapobs.search(query=query)

        for spec in res:
            if fil(spec):
                dpids.append(spec['dp_id'])
                
    return np.array(dpids)

In [ ]:
def download_spectra( dp_ids, folder ):
    for dp_id in tqdm(dp_ids, desc = 'Downloading Data Products'):
        url = "http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?%s&eso_download=file" %dp_id
        response = requests.get(url, stream=True)
        with open("%s/%s.fits" %(folder, dp_id), "wb") as file:
            for data in response.iter_content():
                file.write(data)

In [ ]:
dp_ids = get_dpids(targets, 5, lambda x: True)

In [ ]:
print(dp_ids)

In [ ]:
folder = '../data/fits_SNR0x5'
download_spectra( dp_ids, folder )

In [ ]:
label_file = '../data/HARPS_masterlist_use.csv'

targets = get_targets(label_file)

print(targets)